---
### 钢管订购和运输问题
> 2000年国赛B题

In [1]:
import numpy as np
import cvxpy as cp
import networkx as nx

#### 1. 运费矩阵的计算模型
(1) 计算铁路任意两点间的最小运输费用

In [2]:
s1 = ['S'+str(i) for i in range(1,8)]
s2 = ['A'+str(i) for i in range(1,16)]
s3 = ['B'+str(i) for i in range(1,18)]
nodes = s1 + s2 + s3
L1 = [('B1','B3',450),('B2','B3',80),('B3','B5',1150),('B5','B8',1100),
      ('B4','B6',306),('B6','B7',195),('S1','B7',20),('S1','B8',202),
      ('S2','B8',1200),('B8','B9',720),('S3','B9',690),('B9','B10',520),
      ('B10','B12',170),('S4','B12',690),('S5','B11',462),('B11','B12',88),
      ('B12','B14',160),('B13','B14',70),('B14','B15',320),('B15','B16',160),
      ('S6','B16',70),('B16','B17',290),('S7','B17',30)]
G1 = nx.Graph()
G1.add_nodes_from(nodes)
G1.add_weighted_edges_from(L1)
d1 = nx.floyd_warshall_numpy(G1)
d1

array([[   0., 1402., 1612., ..., 2092., 2252., 2542.],
       [1402.,    0., 2610., ..., 3090., 3250., 3540.],
       [1612., 2610.,    0., ..., 1860., 2020., 2310.],
       ...,
       [2092., 3090., 1860., ...,    0.,  160.,  450.],
       [2252., 3250., 2020., ...,  160.,    0.,  290.],
       [2542., 3540., 2310., ...,  450.,  290.,    0.]])

In [3]:
c1 = np.ones(d1.shape)*np.inf
c1[d1==0] = 0
c1[(d1>0) & (d1<=300)]=20; c1[(d1>300) & (d1<=350)]=23; 
c1[(d1>350) & (d1<=400)]=26; c1[(d1>400) & (d1<=450)]=29; 
c1[(d1>450) & (d1<=500)]=32; c1[(d1>500) & (d1<=600)]=37; 
c1[(d1>600) & (d1<=700)]=44; c1[(d1>700) & (d1<=800)]=50; 
c1[(d1>800) & (d1<=900)]=55; c1[(d1>900) & (d1<=1000)]=60;
ind = (d1>1000) & (d1<np.inf)
c1[ind] = 60 + 5*np.ceil((d1[ind]-1000)/100)
c1

array([[  0.,  85.,  95., ..., 115., 125., 140.],
       [ 85.,   0., 145., ..., 165., 175., 190.],
       [ 95., 145.,   0., ..., 105., 115., 130.],
       ...,
       [115., 165., 105., ...,   0.,  20.,  29.],
       [125., 175., 115., ...,  20.,   0.,  20.],
       [140., 190., 130., ...,  29.,  20.,   0.]])

(2) 构造公路费用的赋权图

In [4]:
L2 = [('A1','A2',104),('A2','B1',3),('A2','A3',301),('A3','B2',2),
      ('A3','A4',750),('A4','B5',600),('A4','A5',606),('A5','B4',10),
      ('A5','A6',194),('A6','B6',5),('A6','A7',205),('A7','B7',10),
      ('S1','A7',31),('A7','A8',201),('A8','B8',12),('A8','A9',680),
      ('A9','B9',42),('A9','A10',480),('A10','B10',70),('A10','A11',300),
      ('A11','B11',10),('A11','A12',220),('A12','B13',10),('A12','A13',210),
      ('A13','B15',62),('A13','A14',420),('S6','A14',110),('A14','B16',30),
      ('A14','A15',500),('A15','B17',20),('S7','A15',20)]
G2 = nx.Graph()
G2.add_nodes_from(nodes)
G2.add_weighted_edges_from(L2)
w2 = nx.to_numpy_array(G2)
d2 = nx.floyd_warshall_numpy(G2)
d2[d2 == 0] = np.Inf
c2 = 0.1*d2
c2

array([[  inf,   inf,   inf, ..., 218.4, 257.2, 306.2],
       [  inf,   inf,   inf, ...,   inf,   inf,   inf],
       [  inf,   inf,   inf, ...,   inf,   inf,   inf],
       ...,
       [218.4,   inf,   inf, ...,   inf,  51.2, 100.2],
       [257.2,   inf,   inf, ...,  51.2,   inf,  55. ],
       [306.2,   inf,   inf, ..., 100.2,  55. ,   inf]])

(3) 计算任意两点间的最小运输费用

In [5]:
c3 = np.minimum(c1, c2) # 对于无法仅使用 1种交通工具互达的两点，c3 == inf
G = nx.Graph(c3)
c = nx.floyd_warshall_numpy(G)  # 重新在整个图中规划路线
c = c[:7, 7:7+15]
c

array([[170.7, 160.3, 140.2,  98.6,  38. ,  20.5,   3.1,  21.2,  64.2,
         92. ,  96. , 106. , 121.2, 128. , 142. ],
       [215.7, 205.3, 190.2, 171.6, 111. ,  95.5,  86. ,  71.2, 114.2,
        142. , 146. , 156. , 171.2, 178. , 192. ],
       [230.7, 220.3, 200.2, 181.6, 121. , 105.5,  96. ,  86.2,  48.2,
         82. ,  86. ,  96. , 111.2, 118. , 132. ],
       [260.7, 250.3, 235.2, 216.6, 156. , 140.5, 131. , 116.2,  84.2,
         62. ,  51. ,  61. ,  76.2,  83. ,  97. ],
       [255.7, 245.3, 225.2, 206.6, 146. , 130.5, 121. , 111.2,  79.2,
         57. ,  33. ,  51. ,  71.2,  73. ,  87. ],
       [265.7, 255.3, 235.2, 216.6, 156. , 140.5, 131. , 121.2,  84.2,
         62. ,  51. ,  45. ,  26.2,  11. ,  28. ],
       [275.7, 265.3, 245.2, 226.6, 166. , 150.5, 141. , 131.2,  99.2,
         76. ,  66. ,  56. ,  38.2,  26. ,   2. ]])

#### 2. 总费用的数学规划模型

In [6]:
l = np.array([104, 301, 750, 606, 194, 205, 201,
              680, 480, 300, 220, 210, 420, 500])
s = np.array([800, 800, 1000, 2000, 2000, 2000, 3000])
p = np.array([160, 155, 155, 160, 155, 150, 160])
min_output = 500
c = c + np.vstack([p]*15).T
c   # 购运费

array([[330.7, 320.3, 300.2, 258.6, 198. , 180.5, 163.1, 181.2, 224.2,
        252. , 256. , 266. , 281.2, 288. , 302. ],
       [370.7, 360.3, 345.2, 326.6, 266. , 250.5, 241. , 226.2, 269.2,
        297. , 301. , 311. , 326.2, 333. , 347. ],
       [385.7, 375.3, 355.2, 336.6, 276. , 260.5, 251. , 241.2, 203.2,
        237. , 241. , 251. , 266.2, 273. , 287. ],
       [420.7, 410.3, 395.2, 376.6, 316. , 300.5, 291. , 276.2, 244.2,
        222. , 211. , 221. , 236.2, 243. , 257. ],
       [410.7, 400.3, 380.2, 361.6, 301. , 285.5, 276. , 266.2, 234.2,
        212. , 188. , 206. , 226.2, 228. , 242. ],
       [415.7, 405.3, 385.2, 366.6, 306. , 290.5, 281. , 271.2, 234.2,
        212. , 201. , 195. , 176.2, 161. , 178. ],
       [435.7, 425.3, 405.2, 386.6, 326. , 310.5, 301. , 291.2, 259.2,
        236. , 226. , 216. , 198.2, 186. , 162. ]])

In [7]:
x = cp.Variable(c.shape, integer=True)
t = cp.Variable(len(c), boolean=True)
y = cp.Variable(len(c[0]), nonneg=True)
z = cp.Variable(len(c[0]), nonneg=True)

obj = cp.Minimize(cp.sum(cp.multiply(c, x)) + 0.05*cp.sum((y**2+y+z**2+z)))
cons = [
    cp.sum(x, axis=1) >= min_output*t,
    cp.sum(x, axis=1) <= cp.multiply(s, t),
    cp.sum(x, axis=0) == y + z,
    x >= 0,
    y[1:] + z[:-1] == l,
    y[0] == 0,
    z[-1] == 0,
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print('最优值为：', prob.value)
print('x：\n', x.value)
print('t：\n', t.value)
print('y：\n', y.value)
print('z：\n', z.value)

最优值为： 1278631.6
x：
 [[ -0.  -0.  -0.  -0. 334. 201. 265.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
   -0.]
 [ -0. 179.  -0.  40. 281.  -0.  -0. 300.  -0.  -0.  -0.  -0.  -0.  -0.
   -0.]
 [ -0.  -0.  -0. 336.  -0.  -0.  -0.  -0. 664.  -0.  -0.  -0.  -0.  -0.
   -0.]
 [ -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
   -0.]
 [ -0.  -0. 508.  92.  -0.  -0.  -0.  -0.  -0. 351. 415.  -0.  -0.  -0.
   -0.]
 [ -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  86. 333. 621.
  165.]
 [ -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
    0.]]
t：
 [1. 1. 1. 0. 1. 1. 0.]
y：
 [ -0. 104. 226. 468. 606. 185. 189. 125. 505. 321. 270.  75. 199. 286.
 165.]
z：
 [  0.  75. 282.  -0.   9.  16.  76. 175. 159.  30. 145.  11. 134. 335.
  -0.]
